In [1]:
import pandas as pd
import re

In [2]:
nuroa = pd.read_csv("../datos_extraidos/casas_nuroa.csv")

In [3]:
# Se tienen algunos valores nulos 
nuroa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             152 non-null    object 
 1   seller            152 non-null    object 
 2   property_type     151 non-null    object 
 3   address           151 non-null    object 
 4   price             151 non-null    float64
 5   bedrooms          147 non-null    float64
 6   bathrooms         145 non-null    float64
 7   built_area        129 non-null    float64
 8   land_area         123 non-null    float64
 9   description       151 non-null    object 
 10  publication_date  151 non-null    object 
dtypes: float64(5), object(6)
memory usage: 13.2+ KB


In [4]:
# Se eliminan los valores nulos
nuroa = nuroa.dropna()
nuroa = nuroa.reset_index(drop="first")

In [5]:
# Se eliminan todas las casas que tienen un remate bancario o hipotecario
# ya que no representan el valor real 
casas_remate = list(nuroa[nuroa.description.str.contains("remate|bancaria|recuperación|hipotecaria", case=False)].index)
nuroa = nuroa.drop(index=casas_remate)
nuroa = nuroa.reset_index(drop="first")

In [6]:
# Adecuo los valores de la columna adddress para hacer una búsqueda efectiva de los zip codes
nuroa["address"] = nuroa["address"].apply(lambda x: x.lower()\
                                                        .replace("á", "a")\
                                                        .replace("é", "e")\
                                                        .replace("í", "i")\
                                                        .replace("ó", "o")\
                                                        .replace("ú", "u"))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace(", oaxaca, oaxaca", ""))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("exhacienda", ""))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("central de", ""))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("fraccionamiento alamos", "fraccionamiento los alamos"))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("res alamos", "residencial alamos"))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("fracc ", "fraccionamiento "))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("2da secc", "segunda seccion"))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("lazaro cardenas 1ra secc", "lazaro cardenas 1a secc"))
nuroa["address"] = nuroa["address"].apply(lambda x: x.replace("unidad hab", "unidad habitacional"))

### Creando la columna zip_code

In [7]:
# Leo la tabla de códigos postales
tabla_CP = pd.read_html("https://codigo-postal.co/mexico/oaxaca/oaxaca-de-juarez/")
tabla_CP = tabla_CP[0]

# Seleccionar las columas que me sirven 
tabla_CP = tabla_CP.iloc[:, :2]
tabla_CP = tabla_CP.drop_duplicates(ignore_index=True)

# Guardar en listas las colonias y sus C.P.
cp = list(tabla_CP.iloc[:, 0])
direccion = list(tabla_CP.iloc[:, 1])

# Crear un diccionario y Eliminar acentos de los nombres de las direcciones
cp_dict = {}
for i, j in zip(direccion, cp):
    cp_dict[i.lower()\
        .replace("á", "a")\
        .replace("é", "e")\
        .replace("í", "i")\
        .replace("ó", "o")\
        .replace("ú", "u")] = j

In [8]:
# Creo una lista con los datos de zip code para cada propiedad
cp_column = []
for i, d in zip(nuroa.index, nuroa.address):
    regex = re.compile(fr"(.{{0,20}}{d}.{{0,20}})", re.IGNORECASE)
    location_name = list(filter(lambda x: regex.findall(x), cp_dict.keys()))

    try:
        cp_column.append(cp_dict[location_name[0]])
    except:
        cp_column.append(None)

In [9]:
# Creo la columna zip_code con los valores obtenidos de la iteración
nuroa["zip_code"] = cp_column

In [10]:
# Se tienen algunos valores nulos en la columna zip_code
nuroa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             86 non-null     object 
 1   seller            86 non-null     object 
 2   property_type     86 non-null     object 
 3   address           86 non-null     object 
 4   price             86 non-null     float64
 5   bedrooms          86 non-null     float64
 6   bathrooms         86 non-null     float64
 7   built_area        86 non-null     float64
 8   land_area         86 non-null     float64
 9   description       86 non-null     object 
 10  publication_date  86 non-null     object 
 11  zip_code          84 non-null     float64
dtypes: float64(6), object(6)
memory usage: 8.2+ KB


In [11]:
# Se eliminan los valores nulos
nuroa = nuroa.dropna(ignore_index=True)

### Creando la columna parking

In [12]:
# Con esta función localizo el texto donde especifica los lugares de estacionamiento en la descripción
def econtrar_textos(texto):
    regex = r"(?i)(.{0,10}estacionamientos?.{0,30}|.{0,10}cocheras?.{0,30}|.{0,10}garages?.{0,30}|.{0,10}parkings?.{0,30})"
    expresion = re.findall(regex, texto)
    expresion = "".join(expresion)
    if len(expresion) > 0:
        valor_de_celda = expresion
    else:
        valor_de_celda = "cero"
    return valor_de_celda

In [13]:
# Función para encontrar en la descripción el número de lugares de estacionamiento
def asignar_valor_estacionamiento(texto):
    # Buscar dígitos numéricos en el texto
    if texto != "cero":    
        numeros_encontrados = re.findall(r'\d+', texto)

        if numeros_encontrados:
            # Tomar el primer número encontrado y convertirlo a entero
            return int(numeros_encontrados[0])
        elif "para dos" in texto.lower():
            return 2
        elif "para tres" in texto.lower():
            return 3
        elif "para cuatro" in texto.lower():
            return 4
        elif "para cinco" in texto.lower():
            return 5
        else:
            return 1

In [14]:
# Aplicar las funciones para buscar los valores de parking
nuroa["parking"] = nuroa["description"].apply(econtrar_textos)
nuroa["parking"] = nuroa["parking"].apply(asignar_valor_estacionamiento)

In [15]:
# Se tienen algunos valores nulos en la columna parking
nuroa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             84 non-null     object 
 1   seller            84 non-null     object 
 2   property_type     84 non-null     object 
 3   address           84 non-null     object 
 4   price             84 non-null     float64
 5   bedrooms          84 non-null     float64
 6   bathrooms         84 non-null     float64
 7   built_area        84 non-null     float64
 8   land_area         84 non-null     float64
 9   description       84 non-null     object 
 10  publication_date  84 non-null     object 
 11  zip_code          84 non-null     float64
 12  parking           65 non-null     float64
dtypes: float64(7), object(6)
memory usage: 8.7+ KB


In [16]:
nuroa.to_csv("datos_nuroa.csv", index=False)